## Retrieve parquet dataset

In [3]:
import requests

url = "https://hugging-face-datasets-api.p.rapidapi.com/first-rows"

querystring = {"split":"train","config":"default","dataset":"back2classroom/sidewalks"}

headers = {
	"X-RapidAPI-Key": "764a64b2dcmsh087dbf31faf8155p11c95ajsnbfdf5914f7ef",
	"X-RapidAPI-Host": "hugging-face-datasets-api.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'dataset': 'back2classroom/sidewalks', 'config': 'default', 'split': 'train', 'features': [{'feature_idx': 0, 'name': 'filename', 'type': {'dtype': 'string', '_type': 'Value'}}, {'feature_idx': 1, 'name': 'tif', 'type': {'dtype': 'binary', '_type': 'Value'}}, {'feature_idx': 2, 'name': 'tfw', 'type': {'dtype': 'binary', '_type': 'Value'}}], 'rows': [{'row_idx': 0, 'row': {'filename': '1001_10_1', 'tif': [255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 255, 219, 0, 67, 0, 8, 6, 6, 7, 6, 5, 8, 7, 7, 7, 9, 9, 8, 10, 12, 20, 13, 12, 11, 11, 12, 25, 18, 19, 15, 20, 29, 26, 31, 30, 29, 26, 28, 28, 32, 36, 46, 39, 32, 34, 44, 35, 28, 28, 40, 55, 41, 44, 48, 49, 52, 52, 52, 31, 39, 57, 61, 56, 50, 60, 46, 51, 52, 50, 255, 219, 0, 67, 1, 9, 9, 9, 12, 11, 12, 24, 13, 13, 24, 50, 33, 28, 33, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50

In [5]:
data = response.json()

# Extract features and rows from the response
features = data['features']
rows = data['rows']

# Create a dictionary to hold the data
data_dict = {}
for feature in features:
    name = feature['name']
    data_dict[name] = []

# Populate the dictionary with row data
for row in rows:
    for feature in row['row']:
        data_dict[feature].append(row['row'][feature])

# Create a DataFrame from the dictionary
df = pd.DataFrame(data_dict)

# Now you have a DataFrame containing the data
print(df)

     filename                                                tif  \
0   1001_10_1  [255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...   
1  1001_10_10  [255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...   
2  1001_10_11  [255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...   
3  1001_10_12  [255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...   
4  1001_10_13  [255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...   
5  1001_10_14  [255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...   
6  1001_10_15  [255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...   
7  1001_10_16  [255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...   
8  1001_10_17  [255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,...   
9  1001_10_18  "/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHB...   

                                                 tfw  
0  [49, 46, 48, 48, 48, 48, 48, 48, 48, 48, 48, 4...  
1  [49, 46, 48, 48, 48, 48, 48, 48, 48, 48, 48, 4...  
2  [49, 46, 48, 48, 48, 48, 48, 48, 48, 48, 48, 4...  
3  [49, 46, 48, 48, 48, 48, 48,

## Download dataset to access locally

In [2]:
mydataset = load_dataset("back2classroom/sidewalks")

Generating train split:   0%|          | 0/180000 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [13]:
mydataset.save_to_disk("images-tiff")

Saving the dataset (0/6 shards):   0%|          | 0/180000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/20000 [00:00<?, ? examples/s]

In [18]:
mydataset_reloaded = load_from_disk("images-tiff")
mydataset_reloaded

DatasetDict({
    train: Dataset({
        features: ['filename', 'tif', 'tfw'],
        num_rows: 180000
    })
    val: Dataset({
        features: ['filename', 'tif', 'tfw'],
        num_rows: 20000
    })
})

## Fine tune

In [1]:
!pip install opencv-python


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tifffile
import os
import leafmap
from patchify import patchify  #Only to handle large images
import random
from scipy import ndimage
import cv2
from samgeo import SamGeo, tms_to_geotiff, get_basemaps

In [2]:
# Load tiff stack images and masks
m = leafmap.Map(center=[40.7434679,-74.175889], zoom = 19) # NJIT
m.add_basemap("SATELLITE")

image = "satellite.tif"
m.user_roi_bounds()
if m.user_roi_bounds() is not None:
    bbox = m.user_roi_bounds()
else:
    bbox = [-95.3704, 29.6762, -95.368, 29.6775]
# tms_to_geotiff(output=image, bbox=bbox, zoom=20, source="Satellite", overwrite=True)

#165 large images as tiff image stack
large_images = tifffile.imread(image)
print(large_images.shape)


(1116, 1790, 3)


In [7]:
# Display the downloaded image on the map.
m.layers[-1].visible = False  # turn off the basemap
m.add_raster(image, layer_name="Image")
m

ImportError: localtileserver is not installed. Please install it before proceeding. https://github.com/banesullivan/localtileserver

In [7]:
#Desired patch size for smaller images and step size.
patch_size = 256
step = 256

all_img_patches = []
for img in range(large_images.shape[0]):
    large_image = large_images[img]
    patches_img = patchify(large_image, (patch_size, patch_size), step=step)  #Step=256 for 256 patches means no overlap

    for i in range(patches_img.shape[0]):
        for j in range(patches_img.shape[1]):

            single_patch_img = patches_img[i,j,:,:]
            all_img_patches.append(single_patch_img)

images = np.array(all_img_patches)

#Let us do the same for masks
all_mask_patches = []
for img in range(large_masks.shape[0]):
    large_mask = large_masks[img]
    patches_mask = patchify(large_mask, (patch_size, patch_size), step=step)  #Step=256 for 256 patches means no overlap

    for i in range(patches_mask.shape[0]):
        for j in range(patches_mask.shape[1]):

            single_patch_mask = patches_mask[i,j,:,:]
            single_patch_mask = (single_patch_mask / 255.).astype(np.uint8)
            all_mask_patches.append(single_patch_mask)

masks = np.array(all_mask_patches)

NameError: name 'patchify' is not defined